### 1. Create The Sales Table Schema

In [0]:
dbutils.widgets.text("catalog_name", "main", "catalog_name")
dbutils.widgets.text("schema_name", "inventory_analytics", "schema_name")
dbutils.widgets.text("lakebase_catalog_name", "reynoldspravindev_inventory_live", "lakebase_catalog_name")
dbutils.widgets.text("lakebase_schema_name", "inventory_app", "lakebase_schema_name")

In [0]:
catalog_name = dbutils.widgets.get('catalog_name')
schema_name = dbutils.widgets.get('schema_name')

lakebase_catalog_name = dbutils.widgets.get('lakebase_catalog_name')
lakebase_schema_name = dbutils.widgets.get('lakebase_schema_name')

In [0]:

# spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog_name}")
spark.sql(f'USE CATALOG {catalog_name}')


spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema_name}")
spark.sql(f"USE SCHEMA {schema_name}")


DataFrame[]

In [0]:
%sql
CREATE  OR REPLACE TABLE store_sales_temp
AS 
SELECT * FROM samples.tpcds_sf1.store_sales

num_affected_rows,num_inserted_rows


In [0]:
spark.sql(f"""CREATE OR REPLACE TABLE items_temp
          AS SELECT DISTINCT item_name from {lakebase_catalog_name}.{lakebase_schema_name}.inventory_items""")

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

### 2. Update Data Surrogate Keys to Current Date

In [0]:
%sql
WITH bounds AS (
  SELECT MIN(d_date) AS old_start, MAX(d_date) AS old_end
  FROM store_sales_temp ss
  JOIN samples.tpcds_sf1.date_dim dd
    ON ss.ss_sold_date_sk = dd.d_date_sk
),
calc AS (
  SELECT old_start, old_end,
        DATEDIFF(DAY, old_start, old_end) AS old_span,
        current_date() - 1  AS new_end
  FROM bounds
)
,date_shifter AS
(
  SELECT DISTINCT dd.d_date_sk AS original_date_sk ,
    dd.d_date AS original_date,
    CAST(DATEADD(
        DAY,
        DATEDIFF(DAY, c.old_start, dd.d_date),
        DATEADD(DAY, -DATEDIFF(DAY, c.old_start, c.old_end), c.new_end)
    ) AS DATE) AS shifted_date
  FROM store_sales_temp ss
      JOIN samples.tpcds_sf1.date_dim dd
      ON ss.ss_sold_date_sk = dd.d_date_sk
  CROSS JOIN calc c
)

MERGE INTO store_sales_temp AS ss
USING (
  SELECT ds.original_date_sk, dd.d_date_sk AS new_date_sk
  FROM date_shifter ds
  JOIN samples.tpcds_sf1.date_dim dd
    ON ds.shifted_date = dd.d_date
) AS src
ON ss.ss_sold_date_sk = src.original_date_sk
WHEN MATCHED THEN
  UPDATE SET ss.ss_sold_date_sk = src.new_date_sk

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
2750838,2750838,0,0


### 2.1 Validate if the update worked as expected

In [0]:
%sql
SELECT max(d_date) , min(d_date)
FROM store_sales_temp ss
JOIN samples.tpcds_sf1.date_dim dd
  ON ss.ss_sold_date_sk = dd.d_date_sk
ORDER BY 1 DESC

max(d_date),min(d_date)
2025-10-21,2020-10-21


In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE store_sales
AS
SELECT DISTINCT d_date AS date, i_category_id as category_id, CAST(FLOOR(rand() * 22) + 1 AS INT) AS warehouse_id,itms.item_name, ss.*
FROM samples.tpcds_sf1.item i
JOIN store_sales_temp ss
  ON ss.ss_item_sk = i.i_item_sk
JOIN samples.tpcds_sf1.date_dim dd
  ON ss.ss_sold_date_sk = dd.d_date_sk
CROSS JOIN items_temp itms
WHERE i_category IS NOT NULL AND i_category_id IS NOT NULL
"""
)

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
%sql
SELECT * FROM store_sales LIMIT 10

date,category_id,warehouse_id,item_name,ss_sold_date_sk,ss_sold_time_sk,ss_item_sk,ss_customer_sk,ss_cdemo_sk,ss_hdemo_sk,ss_addr_sk,ss_store_sk,ss_promo_sk,ss_ticket_number,ss_quantity,ss_wholesale_cost,ss_list_price,ss_sales_price,ss_ext_discount_amt,ss_ext_sales_price,ss_ext_wholesale_cost,ss_ext_list_price,ss_ext_tax,ss_coupon_amt,ss_net_paid,ss_net_paid_inc_tax,ss_net_profit
2020-10-22,7,5,Dining Table,2459145,53853,8908,58421,1255799,5977,17755,7,128,121534,40,18.81,22.00,2.20,0.00,88.00,752.40,880.00,5.28,0.00,88.00,93.28,-664.40
2020-10-22,4,11,Baby Monitor,2459145,49868,9830,20938,1039575,1059,44896,7,135,147369,64,65.88,84.32,5.05,193.92,323.20,4216.32,5396.48,0.00,193.92,129.28,129.28,-4087.04
2020-10-22,7,16,Gaming Console,2459145,33069,10774,61497,989375,6500,38825,2,216,157816,90,98.23,194.49,171.15,0.00,15403.50,8840.70,17504.10,0.00,0.00,15403.50,15403.50,6562.80
2020-10-24,1,16,Car Wax,2459147,48441,1045,7305,1544320,982,11597,1,144,100340,77,23.03,40.30,11.68,0.00,899.36,1773.31,3103.10,62.95,0.00,899.36,962.31,-873.95
2020-10-21,9,1,Protein Powder,2459144,65919,7984,2926,1814838,797,15909,4,144,34097,15,68.73,123.02,40.59,0.00,608.85,1030.95,1845.30,48.70,0.00,608.85,657.55,-422.10
2020-10-22,7,12,Educational Toy Set,2459145,47159,17522,64295,1739497,5899,39101,2,154,195553,76,35.08,59.63,39.95,0.00,3036.20,2666.08,4531.88,60.72,0.00,3036.20,3096.92,370.12
2020-10-22,6,2,Guitar,2459145,39212,11116,17635,null,null,5703,10,284,195680,62,null,64.53,null,null,1440.26,3884.92,4000.86,null,null,null,1569.88,-2444.66
2020-10-25,10,18,Documentary Series,2459148,43507,10244,68666,1291779,6892,7605,4,129,122879,70,60.05,109.29,63.38,0.00,4436.60,4203.50,7650.30,177.46,0.00,4436.60,4614.06,233.10
2020-10-21,4,4,Notebook,2459144,38633,12734,51071,1469887,6750,42766,7,27,223758,87,83.15,149.67,79.32,0.00,6900.84,7234.05,13021.29,276.03,0.00,6900.84,7176.87,-333.21
2020-10-22,3,10,Tablet,2459145,47988,7366,91418,1393080,3895,21557,10,194,43268,80,72.10,105.98,75.24,0.00,6019.20,5768.00,8478.40,481.53,0.00,6019.20,6500.73,251.20


In [0]:
%sql
DROP TABLE IF EXISTS store_sales_temp;
DROP TABLE IF EXISTS items_temp